In [1]:
# -*- coding: utf-8 -*-
# ==========================================
# Real-time Sign Language Recognition Pipeline
# 修正版（LSTM_RNNモデルを構造再定義＋重み読み込み）
# ==========================================

import cv2
import numpy as np
import os
import mediapipe as mp
import json
import glob
import pandas as pd
import datetime
import tensorflow as tf
from tensorflow.keras.models import load_model




# ============================================================
# --- カスタムモデルクラス定義 ---
# ============================================================
# ============================================================
# --- カスタムモデルクラス定義（build対応版） ---
# ============================================================
class LSTM_RNN(tf.keras.Model):
    def __init__(self, n_input, n_hidden, n_classes, **kwargs):
        super(LSTM_RNN, self).__init__(**kwargs)
        self.n_input = n_input
        self.n_hidden = n_hidden
        self.n_classes = n_classes

        self.input_dense = tf.keras.layers.Dense(n_hidden, activation='relu')
        self.lstm1 = tf.keras.layers.LSTM(n_hidden, return_sequences=True)
        self.lstm2 = tf.keras.layers.LSTM(n_hidden)
        self.out = tf.keras.layers.Dense(n_classes, activation='softmax')

    def call(self, x):
        x = self.input_dense(x)
        x = self.lstm1(x)
        x = self.lstm2(x)
        return self.out(x)

    def build(self, input_shape):
        # input_shape = (batch, timesteps, features)
        dummy = tf.zeros(input_shape)
        _ = self.call(dummy)
        super().build(input_shape)


# ============================================================
# --- 設定 ---
# ============================================================

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'  # GPU無効化（必要に応じて変更）

MODEL_PATH = r"C:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\weights\model.h5"

LABELS = [
    "ageru", "understand", "annsinnsuru", "heavy"
]

BASE_OUTPUT_DIR = os.path.join(os.getcwd(), "Inference_Pipeline")
SEQUENCE_LENGTH = 30
N_HIDDEN = 34

# ============================================================
# --- STEP 1: 録画 ---
# ============================================================
def record_video(save_dir, sequence_length):
    print("--- [STEP 1] リアルタイム録画 ---")
    print("ウィンドウにフォーカスを合わせてください。")
    print("  's'キー: 録画開始")
    print("  'e'キー: 録画停止")
    print("  'q'キー: プログラム終了")

    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("エラー: カメラを開けませんでした。")
        return None

    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = 30.0
    video_writer = None
    is_recording = False
    saved_filepath = None

    os.makedirs(save_dir, exist_ok=True)

    while True:
        ret, frame = cap.read()
        if not ret:
            print("エラー: フレームを読み取れませんでした。")
            break

        display_frame = frame.copy()

        if is_recording:
            cv2.circle(display_frame, (40, 40), 15, (0, 0, 255), -1)
            cv2.putText(display_frame, 'REC', (70, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            if video_writer:
                video_writer.write(frame)

        cv2.imshow('Real-time Video Recorder', display_frame)
        key = cv2.waitKey(1) & 0xFF

        if key == ord('s') and not is_recording:
            is_recording = True
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"rec_{timestamp}.mp4"
            saved_filepath = os.path.join(save_dir, filename)
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            video_writer = cv2.VideoWriter(
                saved_filepath, fourcc, fps, (frame_width, frame_height))
            print(f"\n[録画開始] -> 保存先: {saved_filepath}")

        elif key == ord('e') and is_recording:
            print("[録画終了]")
            is_recording = False
            if video_writer:
                video_writer.release()
                video_writer = None
            break

        elif key == ord('q'):
            print("プログラムを終了します。")
            if video_writer:
                video_writer.release()
            saved_filepath = None
            break

    cap.release()
    cv2.destroyAllWindows()
    return saved_filepath


# ============================================================
# --- STEP 2: MediaPipe骨格座標抽出 ---
# ============================================================
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils


def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results


def extract_keypoints(results):
    keypoints = {}
    keypoints["left_hand"] = [{"id": i, "x": res.x, "y": res.y, "z": res.z}
                              for i, res in enumerate(results.left_hand_landmarks.landmark)] if results.left_hand_landmarks else []
    keypoints["right_hand"] = [{"id": i, "x": res.x, "y": res.y, "z": res.z}
                               for i, res in enumerate(results.right_hand_landmarks.landmark)] if results.right_hand_landmarks else []
    return keypoints


def extract_landmarks_from_video(video_path, json_output_dir_base, sequence_length):
    print("\n--- [STEP 2] MediaPipeによる骨格座標の抽出 ---")
    if not os.path.exists(video_path):
        print(f"エラー: 動画が見つかりません: {video_path}")
        return None

    video_name = os.path.splitext(os.path.basename(video_path))[0]
    video_json_dir = os.path.join(json_output_dir_base, video_name)
    os.makedirs(video_json_dir, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    print(f"処理中の動画: {video_path}")

    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        for frame_num in range(sequence_length):
            ret, frame = cap.read()
            if not ret:
                print("警告: フレームが不足しています。")
                break

            _, results = mediapipe_detection(frame, holistic)
            keypoints = extract_keypoints(results)

            with open(os.path.join(video_json_dir, f"{frame_num}.json"), 'w') as f:
                json.dump(keypoints, f, indent=4)

    cap.release()
    cv2.destroyAllWindows()
    print(f"JSONファイルの保存が完了しました: {video_json_dir}")
    return video_json_dir


# ============================================================
# --- STEP 3: JSON→TXT変換 ---
# ============================================================
def convert_json_to_txt(json_dir, txt_output_dir):
    print("\n--- [STEP 3] JSONから単一TXTファイルへ変換 ---")
    os.makedirs(txt_output_dir, exist_ok=True)
    video_name = os.path.basename(json_dir)
    output_path = os.path.join(txt_output_dir, f"{video_name}.txt")

    NUM_KEYPOINTS_PER_HAND = 21
    json_files = sorted(glob.glob(os.path.join(json_dir, "*.json")),
                        key=lambda x: int(os.path.splitext(os.path.basename(x))[0]))

    frames_data = []
    for jf in json_files:
        with open(jf) as f:
            data = json.load(f)
        frame_data = []

        for hand in ["left_hand", "right_hand"]:
            if data.get(hand):
                for kp in data[hand]:
                    frame_data.extend([kp["x"], kp["y"]])
            else:
                frame_data.extend([0.0] * (NUM_KEYPOINTS_PER_HAND * 2))

        frames_data.append(frame_data)

    with open(output_path, "w") as out:
        for frame in frames_data:
            out.write(",".join(map(str, frame)) + "\n")

    print(f"TXTファイルへの変換が完了しました: {output_path}")
    return output_path


# ============================================================
# --- STEP 4: X_val.txt生成 ---
# ============================================================
def prepare_data_for_lstm(source_txt_path, final_output_path, num_steps):
    print("\n--- [STEP 4] 推論用データ(X_val.txt)の作成 ---")
    df = pd.read_csv(source_txt_path, header=None)
    df.interpolate(inplace=True)
    df.fillna(0.0, inplace=True)

    num_frames = len(df)
    if num_frames < num_steps:
        print("エラー: フレーム数が不足しています。")
        return False

    sequences = [df.iloc[:num_steps].values]
    os.makedirs(os.path.dirname(final_output_path), exist_ok=True)
    with open(final_output_path, 'w') as f:
        for seq in sequences:
            np.savetxt(f, seq, delimiter=',')

    print(f"1個のシーケンスを生成し、推論用ファイルに保存しました: {final_output_path}")
    return True


# ============================================================
# --- STEP 5: 推論（修正版） ---
# ============================================================
# ============================================================
# --- STEP 5: 推論（修正版） ---
# ============================================================
def run_inference(x_val_path, model_path, labels, n_steps, n_hidden):
    print("\n--- [STEP 5] LSTMモデルによる推論 ---")

    try:
        # --- データ読み込み ---
        X_ = np.loadtxt(x_val_path, delimiter=',', dtype=np.float32)
        X_val = np.expand_dims(X_, axis=0)  # shape: (1, frame数, feature数)
        n_input = X_val.shape[2]
        n_classes = len(labels)

        # --- モデル読み込み（構造ごと） ---
        model = tf.keras.models.load_model(
            model_path,
            custom_objects={'LSTM_RNN': LSTM_RNN}
        )

        print(f"モデルを正常に読み込みました: {model_path}")
        model.summary()

        # --- 推論 ---
        preds = model.predict(X_val)
        pred_index = np.argmax(preds, axis=1)[0]
        pred_label = labels[pred_index]

        print(f"\n--- [最終推論結果] ---")
        print(f"👉 録画された手話の動作は『 {pred_label} 』です。")

    except Exception as e:
        print(f"モデルの読み込みまたは推論中にエラーが発生しました: {e}")



    # 推論
    try:
        preds = model.predict(X_val)
        pred_index = np.argmax(preds, axis=1)[0]
        pred_label = labels[pred_index]
        print(f"\n--- [最終推論結果] ---")
        print(f"👉 録画された手話の動作は『 {pred_label} 』です。")
    except Exception as e:
        print(f"推論中にエラーが発生しました: {e}")


# ============================================================
# --- 実行部 ---
# ============================================================
if __name__ == '__main__':
    video_dir = os.path.join(BASE_OUTPUT_DIR, "recorded_videos")
    json_dir = os.path.join(BASE_OUTPUT_DIR, "json_output")
    txt_dir = os.path.join(BASE_OUTPUT_DIR, "txt_converted")
    final_dir = os.path.join(BASE_OUTPUT_DIR, "final_input")
    x_val_path = os.path.join(final_dir, "X_val.txt")

    for d in [video_dir, json_dir, txt_dir, final_dir]:
        os.makedirs(d, exist_ok=True)

    recorded = record_video(video_dir, SEQUENCE_LENGTH)
    if recorded:
        json_path = extract_landmarks_from_video(recorded, json_dir, SEQUENCE_LENGTH)
        txt_path = convert_json_to_txt(json_path, txt_dir)
        if prepare_data_for_lstm(txt_path, x_val_path, SEQUENCE_LENGTH):
            run_inference(x_val_path, MODEL_PATH, LABELS, SEQUENCE_LENGTH, N_HIDDEN)
    else:
        print("録画がキャンセルされました。")


--- [STEP 1] リアルタイム録画 ---
ウィンドウにフォーカスを合わせてください。
  's'キー: 録画開始
  'e'キー: 録画停止
  'q'キー: プログラム終了

[録画開始] -> 保存先: c:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\Inference_Pipeline\recorded_videos\rec_20251011_112943.mp4
[録画終了]

--- [STEP 2] MediaPipeによる骨格座標の抽出 ---
処理中の動画: c:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\Inference_Pipeline\recorded_videos\rec_20251011_112943.mp4
JSONファイルの保存が完了しました: c:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\Inference_Pipeline\json_output\rec_20251011_112943

--- [STEP 3] JSONから単一TXTファイルへ変換 ---
TXTファイルへの変換が完了しました: c:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\Inference_Pipeline\txt_converted\rec_20251011_112943.txt

--- [STEP 4] 推論用データ(X_val.txt)の作成 ---
1個のシーケンスを生成し、推論用ファイルに保存しました: c:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\Inference_Pipeline\final_input\X